### RatJ Day3 two novel recording info

In [ ]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from neuropy import plotting
from neuropy import analyses

sess = subjects.tn.ratJday3[0]
print(sess.recinfo)

### Probe config
- Copied directly from ratJDay2NSD

In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=4,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plot_probe(prbgrp)

In [ ]:
from neuropy.io import SpykingCircusIO

SpykingCircusIO.write_probe_file(
    sess.filePrefix.with_suffix(".prb"),
    sess.probegroup,
    rmv_badchans=True,
    combine_shanks=True,
)

### Experimental paradigm

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        # "start": [0, 11510, 14820,19200,22980], # These numbers may not be correct
        # "stop": [11400, 14693, 19140,22860,54130],
        "label": ["pre", "maze1", "post1",'maze2','post2'],
    }
)

Epoch(epochs=epochs).save(sess.filePrefix.with_suffix(".paradigm.npy"))

### Artifact epochs

In [ ]:
signal = sess.eegfile.get_signal([63])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=1.5,edge_cutoff=1)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))

In [ ]:
signal = sess.eegfile.get_signal([63])
plotting.plot_artifact_epochs(artifact_epochs, signal)

In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix('.dead')
SpykingCircusIO.write_epochs(file,sess.artifact)